In [3]:
# Import neccessary packages 
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
pn.extension('tabulator')

In [5]:
# load data
df = pd.read_excel("SoftdrinkconsumptionpercapitaQSD(litrespersonyear)Cleaned.xlsx")
df.head()

,Country,Continent,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Algeria,Africa,32.128171,34.445612,36.153728,37.787392,39.341716,40.937912,42.363432,43.450741,44.435014,45.349640,46.079144,46.752539,47.173792,47.517089,47.793123
1,Angola,Africa,8.295544,9.099236,9.994191,10.871929,12.152194,12.833377,13.776878,14.607022,14.968888,15.753421,16.229483,16.904189,17.469581,18.054229,18.690265
2,Argentina,South America,184.517875,188.850281,187.710753,182.397618,168.192707,158.933968,155.530925,162.489705,166.953843,162.692340,162.790602,151.328342,150.169372,139.963081,132.806281
3,Australia,Oceania,150.590901,150.676943,150.576336,146.158886,145.266106,145.056716,143.181932,141.307865,138.720577,135.577342,132.244506,128.422328,124.708380,122.373489,120.025964
4,Austria,Europe,135.505660,133.590925,133.187395,132.757319,131.933044,130.546163,128.470062,129.021514,126.961152,122.211197,122.139218,121.228688,119.390522,120.062890,120.728378


In [6]:
# Creat a Dropdown manu for filtering data based on continents 
def get_continent_names():
    return sorted(list(df.Continent.unique()))

# Create selection widget 
select_widget = pn.widgets.Select(options=get_continent_names())

In [7]:


# Query dataframe based on value provided in dropdown manu
def get_data_by_continent(continent_name):
    return df[df.Continent==continent_name]

# Based on the selection display data table on the page 
@pn.depends(select_widget)
def continent_tabular_widget(continent_name):
     return pn.widgets.Tabulator(get_data_by_continent(continent_name), width=500)


In [8]:
# Compute average for each continent and prepare the date to be plotted 
def get_mean_by_continent(continent_name):
    data=get_data_by_continent(continent_name).drop(columns=["Country"]).groupby("Continent").mean()
    df_melt = data.melt(var_name='Year', value_name='value')
    return df_melt

# Dispaly line charts based on chosen continent
@pn.depends(select_widget)
def continent_mean_line_chart_widget(continent_name):
    data= get_mean_by_continent(continent_name) 
    plot = data.hvplot.line(x='Year', y='value')
    plot.opts(ylabel="Average Liters/Person/Year")
    plot.opts(title = f"Average Soft Drinks Consumption for {continent_name} 2005 - 20019")
    return pn.panel(plot)


In [9]:
# Query top 10 country in each continent based on their average value 
def get_top10_by_continent(continent_name):
    data = get_data_by_continent(continent_name)
    data_copy = data.copy()
    data_copy['Average'] =data_copy.select_dtypes(include='number').mean(axis=1)
    return data_copy.nlargest(10,"Average")

# Create a bar chart to diplay top 10 country in each continent 
@pn.depends(select_widget)
def country_bar_chart_widget(continent_name):
    df_sorted = get_top10_by_continent(continent_name).sort_values(by='Average')
    plot = df_sorted.hvplot(x="Country",y="Average",
                            kind="bar", 
                            color="goldenrod",
                            title=f"Top Soft Drinks consumer Countries in {continent_name}")
    plot.opts(invert_axes=True)
    plot.sort()
    plot.opts(ylabel="Yearly Average Liters/Person/Year")
    return pn.panel(plot)

In [10]:
# Add image
image1 = pn.pane.Image("softdrinks .jpg",width=300, height=200)

In [ ]:
template = pn.template.MaterialTemplate(
    title='Grobal Soft Drinks consumption',
)

template.main.append(
    pn.Row(
        pn.Column(select_widget,continent_tabular_widget),
        pn.Column(continent_mean_line_chart_widget,country_bar_chart_widget),
    )
)

pn.serve(template)

Launching server at http://localhost:63462
